# Part 1

In [1]:
INPUT_FILE = "day4.txt"

In [2]:
with open(INPUT_FILE) as f:
    loglines = [s.strip() for s in f.readlines()]

In [3]:
loglines[:5]

['[1518-03-31 00:26] falls asleep',
 '[1518-04-03 00:38] wakes up',
 '[1518-11-21 00:14] falls asleep',
 '[1518-07-21 00:34] wakes up',
 '[1518-04-02 00:20] falls asleep']

In [4]:
from collections import namedtuple, defaultdict
from datetime import datetime

In [5]:
ASLEEP = "falls asleep"
WAKE_UP = "wakes up"

In [6]:
Event = namedtuple("Event", "time event")
Interval = namedtuple("Interval", "start end activity")

In [7]:
def parse(s):
    time_str, event = s.split("]")
    time_str = time_str[1:]
    event = event.strip()
    return Event(datetime.strptime(time_str, "%Y-%m-%d %H:%M"), event)

def get_guard_id(s):
    _, gid, _, _ = s.split()
    return gid

### Parse log lines into events

In [8]:
events = list(sorted((parse(s) for s in loglines)))

In [9]:
events[:5]

[Event(time=datetime.datetime(1518, 3, 19, 0, 2), event='Guard #2953 begins shift'),
 Event(time=datetime.datetime(1518, 3, 19, 0, 37), event='falls asleep'),
 Event(time=datetime.datetime(1518, 3, 19, 0, 41), event='wakes up'),
 Event(time=datetime.datetime(1518, 3, 20, 0, 0), event='Guard #457 begins shift'),
 Event(time=datetime.datetime(1518, 3, 20, 0, 14), event='falls asleep')]

### Create Activity Intervals for each guard

In [10]:
guard_events = defaultdict(list)

current_guard = None
sleep_start = 0
prev_time = 0

for time, event in events:
    if event == ASLEEP:
        # Assuming guard doesn't sleep twice without waking, i.e. valid events
        interval = Interval(prev_time, time, "awake")
        guard_events[current_guard].append(interval)
    elif event == WAKE_UP:
        # Record current guard as asleep from last time until now
        interval = Interval(prev_time, time, "asleep")
        guard_events[current_guard].append(interval)
    else:
        # Record current guard as awake from last known time till now
        interval = Interval(prev_time, time, "awake")
        guard_events[current_guard].append(interval)
        
        # Change of guard
        current_guard = get_guard_id(event)
        
    prev_time = time
    
del guard_events[None]

In [11]:
def get_total_sleep_time(intervals):
    sleep_intervals = [i for i in intervals if i.activity == "asleep"]
    times = [(i.end - i.start).total_seconds() for i in sleep_intervals]
    return sum(times)

### Get total sleep time per guard, and find the guard who slept the most

In [12]:
max_sleep = 0
max_sleep_guard = None

for guard, intervals in guard_events.items():
    this_guard_slept_for = get_total_sleep_time(intervals)
    if this_guard_slept_for > max_sleep:
        max_sleep = this_guard_slept_for
        max_sleep_guard = guard

In [13]:
max_sleep_guard, max_sleep

('#2953', 27900.0)

In [14]:
def most_asleep(intervals):
    # Frequency map for each minute of the midnight hour
    sleep_times = [0]*60
    
    # For every minute that this guard was asleep
    # Increase the frequency of that minute
    for i in intervals:
        if i.activity != "asleep":
            continue

        for j in range(i.start.minute, i.end.minute):
            sleep_times[j] += 1

    # Find the minute with maximum frequency
    return max(enumerate(sleep_times), key=lambda k: k[1])

In [15]:
# Find the minute for which this guard was asleep the most
most_asleep(guard_events["#2953"])

(39, 16)

In [16]:
39*2953

115167

# Part 2

`guard_sleeps` is a dictionary of frequency lists. Each outer key is the guard ID, and the inner list is a frequency map for each minute of the midnight hour

In [17]:
guard_sleeps = defaultdict(lambda: [0]*60)

In [18]:
max_sleep_minute = -1
max_sleep_freq = 0
max_guard = None

for guard, intervals in guard_events.items():
    max_sleep_minute_guard, max_sleep_freq_guard = most_asleep(intervals)
    
    if max_sleep_freq_guard > max_sleep_freq:
        max_sleep_freq = max_sleep_freq_guard
        max_sleep_minute = max_sleep_minute_guard
        max_guard = guard

In [19]:
max_guard, max_sleep_minute

('#1069', 30)

In [20]:
1069*30

32070